# Part 3: Advanced Analysis

In this part, we will implement advanced analysis techniques for physiological time series data, including time-domain feature extraction, frequency analysis, and wavelet transforms.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
import pywt
import os

# Set plotting style
plt.style.use('seaborn')
sns.set_context('notebook')

C:\Users\wammi\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\wammi\AppData\Local\Temp\ipykernel_84148\1751286746.py:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
def extract_time_domain_features(data, window_size=60):
    """Extract time-domain features from physiological signals.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    window_size : int, optional
        Size of the rolling window in seconds, default=60
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing extracted features for each window
    """
    # Convert window_size from seconds to number of samples
    # Assuming data is sampled at 1 Hz (1 sample per second)
    window_samples = window_size
    
    # Initialize DataFrame for features
    features = pd.DataFrame()
    
    # Basic statistics using rolling window
    features['mean'] = data['heart_rate'].rolling(window=window_samples).mean()
    features['std'] = data['heart_rate'].rolling(window=window_samples).std()
    features['min'] = data['heart_rate'].rolling(window=window_samples).min()
    features['max'] = data['heart_rate'].rolling(window=window_samples).max()
    
    # Heart rate statistics
    features['mean_hr'] = data['heart_rate'].rolling(window=window_samples).mean()
    features['std_hr'] = data['heart_rate'].rolling(window=window_samples).std()
    
    # Beat-to-beat variability
    rr_intervals = 60000 / data['heart_rate']  # Convert HR to RR intervals in ms
    
    # Calculate successive differences within each window
    rr_diff = rr_intervals.diff()
    
    # RMSSD (Root Mean Square of Successive Differences)
    features['rmssd'] = np.sqrt(rr_diff.rolling(window=window_samples).apply(lambda x: np.mean(x**2)))
    
    # SDNN (Standard Deviation of NN intervals)
    features['sdnn'] = rr_intervals.rolling(window=window_samples).std()
    
    # pNN50 (Percentage of successive RR intervals differing by >50ms)
    features['pnn50'] = rr_diff.rolling(window=window_samples).apply(
        lambda x: 100 * np.sum(np.abs(x) > 50) / len(x) if len(x) > 0 else 0
    )
    
    # Drop NaN values from rolling window calculations
    features = features.dropna()
    
    return features

In [3]:
def analyze_frequency_components(data, sampling_rate, window_size=60):
    """Perform frequency-domain analysis on physiological signals.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    sampling_rate : float
        Sampling rate of the signal in Hz
    window_size : int, optional
        Size of the analysis window in seconds, default=60
        
    Returns:
    --------
    dict
        Dictionary containing frequency analysis results
    """
    # Convert window_size from seconds to number of samples
    window_samples = int(window_size * sampling_rate)
    
    # Initialize results dictionary
    results = {}
    
    # Process data in windows
    n_windows = len(data) // window_samples
    all_frequencies = []
    all_power = []
    
    for i in range(n_windows):
        window_data = data['heart_rate'].iloc[i*window_samples:(i+1)*window_samples]
        
        # Calculate PSD using Welch's method
        frequencies, power = signal.welch(
            window_data,
            fs=sampling_rate,
            nperseg=window_samples
        )
        
        all_frequencies.append(frequencies)
        all_power.append(power)
    
    # Average results across windows
    results['frequencies'] = np.mean(all_frequencies, axis=0)
    results['power'] = np.mean(all_power, axis=0)
    
    # Define frequency bands
    bands = {
        'VLF': (0.003, 0.04),
        'LF': (0.04, 0.15),
        'HF': (0.15, 0.4)
    }
    
    
    # Calculate power in each band
    results['bands'] = {}
    for band_name, (low, high) in bands.items():
        mask = (results['frequencies'] >= low) & (results['frequencies'] <= high)
        results['bands'][band_name] = np.sum(results['power'][mask])
    
    # Calculate LF/HF ratio
    results['bands']['LF/HF'] = results['bands']['LF'] / results['bands']['HF']
    
    return results

In [10]:
def analyze_time_frequency_features(data, sampling_rate, window_size=60):
    """Analyze time-frequency features using wavelet transforms.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    sampling_rate : float
        Sampling rate of the signal in Hz
    window_size : int, optional
        Size of the analysis window in seconds, default=60
        
    Returns:
    --------
    dict
        Dictionary containing time-frequency analysis results
    """
    # Convert window_size from seconds to number of samples
    window_samples = int(window_size * sampling_rate)
    
    # Initialize results dictionary
    results = {}
    
    # Define wavelet scales
    scales = np.arange(1, 128)
    results['scales'] = scales
    
    # Process data in windows
    n_windows = len(data) // window_samples
    all_coefficients = []
    all_energy = []
    
    for i in range(n_windows):
        window_data = data['heart_rate'].iloc[i*window_samples:(i+1)*window_samples]
        
        # Apply continuous wavelet transform
        coefficients, frequencies = pywt.cwt(
            window_data,
            scales,
            'morl',
            sampling_period=1/sampling_rate
        )
        
        all_coefficients.append(coefficients)
        all_energy.append(np.abs(coefficients)**2)
    
    # Average results across windows
    results['coefficients'] = np.mean(all_coefficients, axis=0)
    results['time_frequency_energy'] = np.mean(all_energy, axis=0)

    # Add dominant frequency components over time
    dominant_frequencies = frequencies[np.argmax(results['time_frequency_energy'], axis=0)]
    results['dominant_frequencies'] = dominant_frequencies

    # Ensure output directory exists
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)

    # Save results as .npz file
    np.savez(os.path.join(output_dir, 'wavelet_results.npz'),
             scales=scales,
             coefficients=results['coefficients'],
             time_frequency_energy=results['time_frequency_energy'],
             dominant_frequencies=dominant_frequencies)
    
    return results